---
title: "Introduction to Numpy"
author: "Vahram Poghosyan"
date: "2023-01-13"
categories: ["Python", "Python Scientific Libraries", "Numpy"]
format:
  html:
    code-fold: true
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../javascript/light-dark.js"></script>
---

# Importing NumPy

[NumPy](https://numpy.org/) is a scientific computing library for Python. It's an extensive collection of pre-written code that optimizes and extends, among other things, the Python array (i.e. `list`) object into an n-dimensional NumPy array called `ndarray`. It comes with a variety of tools, such as matrix operations and common mathematical functions, that enable Python to perform complex mathematical tasks such as solve linear algebraic problems, generate pseudo-random numbers, perform Fourier analysis, etc.

We import NumPy, as we import any other library, using the `import` keyword (with or without a shorthand).

```python
import numpy
```
 Or, alternatively:
 
```python
import numpy as np
```

# Optimizations

As we've briefly discussed in the ["Python for Machine Learning - Pandas"](https://v-poghosyan.github.io/blog/python%20for%20ml/pandas/machine%20learning/2021/12/28/Python-for-ML-Pandas.html#Broadcasted-and-Vectorized-Operations) post, NumPy works by delegating tasks to well-optimized C code under the hood. In this way it exploits the flexibility of Python while bypassing its speed limitations as an *interpreted language* and, instead, exploiting the speed advantages of a *compiled language*.


## Scalable Memory Representation

One of the things NumPy optimizes is data storage. 
In contrast to Python 3.x's scalable memory representation of numeric values, such as integers, which can grow to accommodate a given number, NumPy stores numeric types in fixed-sized blocks of memory (e.g. `int32` or `int64`). This means NumPy is able to take advantage of the low-level CPU instructions of modern processors that are designed for fixed-sized numeric types. Another advantage of fixed-sized storage is that consecutive blocks of memory can be allocated, which enables the libraries upon which NumPy relies to do extremely performant computations. This enforcement of fixed-sized data types is part of the  optimization strategy NumPy uses called *vectorization*.

## Vectorization

As already discussed in the [aforementioned post](https://v-poghosyan.github.io/blog/python%20for%20ml/pandas/machine%20learning/2021/12/28/Python-for-ML-Pandas.html#Broadcasted-and-Vectorized-Operations), vectorization is the process by which NumPy stores an array internally in a contiguous block of memory, and restricts its contents to only one data type. Letting Python know this data type in advance, NumPy can then skip the per-iteration type checking that Python normally does as its iterating through a loop in order to speed up our code. Optimizing the array data structure in such a way enables NumPy to delegate most of the operations on such arrays to pre-written C code under the hood. In effect, this simply means that looping occurs in C instead of Python.

## Broadcasting

The term *broadcasting* describes the process by which NumPy performs arithmetic operations on arrays of different dimensions. The process is usually as follows: the smaller array is “broadcast” across the larger array so that the two arrays have compatible dimensions. Broadcasting provides a means of vectorizing array operations. 

## Comparing Runtime
To demonstrate the performance optimizations of NumPy, let's compare squaring every element of a `1,000,000`-element array and summing the results. 

### Using a Python List

First, we will use a Python list:

In [1]:
unoptimized_list = list(range(1000000))

Squaring each element and summing:

In [2]:
import numpy as np
%timeit np.sum([i**2 for i in unoptimized_list])

342 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


> Note: Even though we're using NumPy's `sum()` method, since the input we're passing to it is a regular Python list, NumPy optimizations are not applied. 

<br>

As we can see the whole thing took about `314 ms`.

### Using a NumPy Array

Now let's do the same with a NumPy array, which also gives us the opportunity to introduce the syntax for defining one using a range.

In [12]:
optimized_array = np.arange(1000000)

Let's check the type of `optimized_array` to convince ourselves that it is, indeed, a NumPy `ndarray`.

In [25]:
type(optimized_array)

numpy.ndarray

Now, finally, let's square each element and sum the results:

In [45]:
%timeit np.sum(optimized_array**2)

1.61 ms ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Remarkably, the run-time was cut from `314 ms` to only around `1.61ms`!

# NumPy Basics

Let's explore some of the ways in which we can represent arrays and matrices in NumPy.

## Creating Arrays

We've already seen how we can create a 1-dimensional NumPy array of consecutive integers $0,1,...,n-1$ using the `arrange()` method. 

The standard way of creating a NumPy array is passing a Python list to the constructor `array()` like so:

In [16]:
a = np.array([1,2,3])
a

array([1, 2, 3])

We can also create some common arrays, such as an array of consecutive integers, with some special methods such as `arange()`, which takes an integer $n$ as input and creates a sequential array from $0,...,n-1$.

In [47]:
np.arange(5) # array([0, 1, 2, 3, 4])

array([0, 1, 2, 3, 4])

## Representing Matrices

Let's represent a $2 \times 3$ matrix 
$
A = \begin{bmatrix}
1 & 2 & 3\\
4 & 5 & 6
\end{bmatrix}
$
using NumPy:

In [7]:
A = np.array([[1,2,3],
              [4,5,6]])
A

array([[1, 2, 3],
       [4, 5, 6]])

There are also ways to quickly create some common matrices using special methods. 

For example, `ones()` accepts a shape pair, and creates a matrix of $1$s with of the given shape.

In [48]:
np.ones((2,3))

array([[1., 1., 1.],
       [1., 1., 1.]])

The method, `zeros()` works the same way as `ones()`:

In [49]:
np.zeros((2,3))

array([[0., 0., 0.],
       [0., 0., 0.]])

Meanwhile, `identity()` accepts an integer $n$ as input and creates a square $n \times n$ identity matrix.

In [50]:
np.identity(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

Creating a matrix with identical elements in general uses the `full()` method which takes a `shape` attribute, a `value` attribute and on optional `dtype` attribute as follows:

In [54]:
np.full((2,3), 7, dtype = int)

array([[7, 7, 7],
       [7, 7, 7]])

## Indexing

Indexing a 1-dimensional NumPy array is done as expected, through the use of the trusty brackets `[]`. Indexing an n-dimensional matrix in NumPy still uses `[]` but it introduces a new, improved, syntax. 

Suppose we'd like to access the element in the first row, and last column of `A`. The standard way would be: 

In [20]:
A[0][2]

3

As we can see, that still works. But the recommended and, subjectively speaking, prettier way is:

In [21]:
A[0,2]

3

Of course, slicing still works as expected.

For example, let's print the entire first row of `A`:

In [22]:
A[0,:]

array([1, 2, 3])

The entire first column: 

In [23]:
A[:,0]

array([1, 4])

Finally, let's print the submatrix $
\begin{bmatrix}
2 & 3
\end{bmatrix}
$:

In [24]:
A[0,1:]

array([2, 3])

## Properties and Methods of NumPy Arrays

A few of the useful properties and methods of `ndarray` are highlighted in this section.


* `shape` - returns the shape of the matrix as an $(m,n)$ pair

    ```python
    A.shape # (2,3)
    ```

* `ndim` - returns the dimension of a matrix as a single digit

    ```python
    A.ndim # 2
    ```
> Note: The output of the `ndim` property should *not* be understood in a linear algebraic sense as the dimension of either the domain or range of the corresponding transformation, nor the dimension of either of its four fundamental subspaces. It is to *only* be understood in the data structure sense as the level of nestedness of the array.

* `size` - returns the total number of elements in the matrix

    ```python
    A.size # 6
    ```

* `dtype` - returns the data type of the elements in the matrix.

    ```python
    A.dtype # dtype('int32')
    ```
> Note: If the `ndarray` does not represent a matrix, such as `B = np.array([[1,2,3],[4,5]])` then `dtype` outputs `O` signifying that the entries are general Python objects. In such a case, the array loses its optimizations. 

### Statistical and Mathematical Methods

There is also a vast selection of statistical, and more generally, mathematical methods that `ndarrays` come with. Here are a few of the common ones:

* `sum()` - returns the sum of all the entries

    ```python
    A.sum() # 21
    ```
    It also accepts an `axis` attribute where `axis = 0` refers to the sum along the columns, and `axis = 1` refers to the sum along the rows.

    ```python
    A.sum(axis = 0) # [5,7,9]
    ```
    ```python
    A.sum(axis = 1) # [6,15]
    ```
* `mean()` - returns the empirical mean of all the entries 

    ```python
    A.mean() # 3.5
    ```
* `var()` - returns the variance of the entries

    ```python
    A.var() # 2.9166666666666665
    ```
* `std()` - returns the standard deviation of the entries

    ```python
    A.std() # 1.707825127659933
    ```

## Multi-Indexing, Filtering, and Broadcasted Operations

Recall from the [Pandas article](https://v-poghosyan.github.io/blog/python%20for%20ml/pandas/machine%20learning/2021/12/28/Python-for-ML-Pandas.html) the ways in which we were able to multi-index and filter, and how we eliminated the need for using Python loops and list comprehensions using broadcasted operators instead. Since both a Pandas `Series` and a `DataFrame` are extensions of NumPy's `ndarray`, all of these apply here as well. 

As a refresher on broadcasted operations, here are a few filtering examples. 

Let's obtain those elements of A that are greater than `3`:

In [38]:
A[A > 3]

array([4, 5, 6])

Now let's obtain those elements of A that are greater than the empirical mean:

In [39]:
A[A > A.mean()]

array([4, 5, 6])

What about those elements of A that are less than or equal to the empirical mean?

In [40]:
A[~(A > A.mean())]

array([1, 2, 3])

Which is equivalent to:

In [41]:
A[A <= A.mean()]

array([1, 2, 3])

## Matrix Operations

One of NumPy's key features is the way in which it simplifies matrix operations in Python. It offers simple syntax to add, multiply, *transpose*, *invert*, flatten, etc.

### Addition

Addition of matrices is, by default, per-element (as are all NumPy operations). There's no special syntax, it's done through the `+` operator.

For example:

In [57]:
A = np.ones((2,3))
B = np.ones((2,3))
A + B

array([[2., 2., 2.],
       [2., 2., 2.]])

### Multiplication

The operator `*` performs per-element multiplication.

In [59]:
A = np.full((2,3), 2, dtype = int)
B = np.full((2,3), 3, dtype = int)
A * B

array([[6, 6, 6],
       [6, 6, 6]])

But this, as we know, isn't matrix multiplication as it's commonly defined in mathematics — that being the inner product of corresponding rows and columns. For instance, if we try to multiply an $m \times n$ matrix with an $n \times p$ matrix, NumPy will throw the following error:

In [11]:
A = np.full((2,3), 2, dtype = int)
B = np.full((3,4), 3, dtype = int)
A * B

```
ValueError: operands could not be broadcast together with shapes (2,3) (3,4) 
```

This is because per-element operations require the shapes of the operands to be the same or compatible by broadcasting. Here NumPy attempts to broadcast one operand to match the shape of the other one, but broadcasting is impossible between matrices of shapes $2 \times 3$ and $3 \times 4$ per [broadcasting rules](https://numpy.org/doc/stable/user/basics.broadcasting.html#general-broadcasting-rules).

There's a workaround that lets us use `*`. Since NumPy overloads the `*` operator, it works as it should for `numpy.matrix` types. 

In [9]:
A = np.matrix([[2,2,2],
               [2,2,2]])
B = np.matrix([[3,3,3,3],
               [3,3,3,3],
               [3,3,3,3]])
A * B

matrix([[18, 18, 18, 18],
        [18, 18, 18, 18]])

However, this is not the recommended way to carry out matrix multiplication in NumPy. Overloaded operators can produce convoluted code. For instance, we may have many different `matrix` and `ndarray` data structures and never be able to tell the outcome of a particular `*` operation. We should avoid such ambiguity whenever possible. 

Instead, the recommended way to do matrix multiplication is through the `@` operator. 

When we use `@` NumPy internally uses its `matmul()` method. So, the following are equivalent and both produce the matrix product of `A` and `B`.

In [61]:
A = np.full((2,3), 2, dtype = int)
B = np.full((3,4), 3, dtype = int)
A @ B

array([[18, 18, 18, 18],
       [18, 18, 18, 18]])

In [62]:
A = np.full((2,3), 2, dtype = int)
B = np.full((3,4), 3, dtype = int)
np.matmul(A,B)

array([[18, 18, 18, 18],
       [18, 18, 18, 18]])

NumPy also offers, `dot()` which, for one and two dimensional matrices, is equivalent to `matmul()`. So, the following is yet another way we can multiply two matrices:

In [10]:
A = np.full((2,3), 2, dtype = int)
B = np.full((3,4), 3, dtype = int)
A.dot(B)

array([[18, 18, 18, 18],
       [18, 18, 18, 18]])

However, `matmul()` is preferred over `dot()` because of the clarity of its name, and because the *dot product* has a distinct mathematical meaning separate from matrix multiplication. 